# 命名实体识别

In [1]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser  # 格式化输出

load_dotenv()
llm = ChatOpenAI(model_name='qwen2:7b', temperature=0)

需要命名实体的识别，是要把实体提取出来，填充CQL的，所以我们希望大模型处理完之后的数据是字典数据，这个过程，我们需要用到LangChain中的格式化输出。

In [2]:
# 定义实体字段
# 类型有：list, string, number
response_schemas = [
    ResponseSchema(type='list', name='disease', description='疾病名称实体'),   # 可随时修改、增加
    ResponseSchema(type='list', name='symptom', description='疾病症状实体'),
    ResponseSchema(type='list', name='drug', description='药物名称实体'),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"disease": list  // 疾病名称实体\n\t"symptom": list  // 疾病症状实体\n\t"drug": list  // 药物名称实体\n}\n```'

有了上面的输出，后续就可以直接从大模型的回复中摘出markdown格式的json了。

但是我们的提示词里面带了注释，比较笨的模型可能会在最后的结果里继续附带注释，导致我们处理的代码报错。

这段提示词会是最后大的提示词中的一小部分。

此外，大模型可能会自己生成一些实体，这不符合我们的需求，需要人为规定只能抽取我们给它的实体。

In [15]:
template = '''
1、从以下用户输入的句子中，提取实体内容。
2、仅根据用户输入抽取，不要推理。
3、注意json格式，在json中不要出现//

{format_instructions}

用户输入：{input}

输出：
'''

prompt = PromptTemplate(
    template=template,
    partial_variables={'format_instructions': format_instructions},
    input_variables=['input']
)

prompt = prompt.format(input='感冒是一种什么病？')
prompt

'\n1、从以下用户输入的句子中，提取实体内容。\n2、仅根据用户输入抽取，不要推理。\n3、注意json格式，在json中不要出现//\n\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":\n\n```json\n{\n\t"disease": list  // 疾病名称实体\n\t"symptom": list  // 疾病症状实体\n\t"drug": list  // 药物名称实体\n}\n```\n\n用户输入：感冒是一种什么病？\n\n输出：\n'

最后我们来测试一下：

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=llm,
    prompt=prompt
)

llm_output = chain.run(input='感冒是一种什么病？会导致咳嗽吗？')
llm_output

'```json\n{\n\t"disease": ["感冒"],\n\t"symptom": ["咳嗽"],\n\t"drug": []\n}\n```'

In [16]:
output = output_parser.parse(llm_output)
print(output, type(output))

{'disease': ['感冒'], 'symptom': ['咳嗽'], 'drug': []} <class 'dict'>
